In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from drawinganalyses.datasets.drawings_pytorch import DrawingModule, DrawingDataset
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory
from torch.utils.data import random_split
import torch

cudnn.benchmark = True
plt.ion()   # interactive mode

import torch.nn.functional as F

from pathlib import Path

#gpu or CPU 
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device= torch.device("cpu")

os.chdir(os.path.expanduser("/home/jfalck/PycharmProjects/drawinganalyses"))
LOCAL_DATA_DIR = Path("/home/jfalck/PycharmProjects/drawinganalyses/drawinganalyses/data")
# Dossier de stockages des modeles
MODEL_DATA_DIR = Path("/home/jfalck/PycharmProjects/drawinganalyses/drawinganalyses/models/Molly")

annotations_file = "labels.csv" 
models_storage = MODEL_DATA_DIR
dataset_name = 'Humans'
label_to_str = {0: 'Children', 1: 'Adult'}
batch_size=4
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.Resize((256, 256))])


/home/jfalck/PycharmProjects/drawinganalyses/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                label = label_to_str[labels[j].item()]
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j]]}, real label : {label}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

os.chdir(os.path.expanduser("/home/jfalck/PycharmProjects/drawinganalyses"))
LOCAL_DATA_DIR = Path("/home/jfalck/PycharmProjects/drawinganalyses/drawinganalyses/data")
# Dossier de stockages des modeles
MODEL_DATA_DIR = Path("/home/jfalck/PycharmProjects/drawinganalyses/drawinganalyses/models/Humans")
PATH = os.path.join(MODEL_DATA_DIR, "train_Humans.pth")
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
label_to_str = {0: 'Children', 1: 'Adult'}
class_names = list(label_to_str.values())
model_ft.fc = nn.Linear(num_ftrs, len(class_names))
#gpu or CPU 
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device= torch.device("cpu")
model_ft = model_ft.to(device)
model_ft.load_state_dict(torch.load(PATH))

model_ft.eval()

/home/jfalck/PycharmProjects/drawinganalyses/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/jfalck/PycharmProjects/drawinganalyses/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
generator1 = torch.Generator().manual_seed(42)
dataset = DrawingDataset(
    dataset_name=dataset_name,
    annotations_file=annotations_file,
    data_dir=LOCAL_DATA_DIR,
    label_to_str=label_to_str,
    transform=transform)
trainset, valset, testset = random_split(dataset, [0.8, 0.1, 0.1], generator=generator1)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=1,
                                          shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valset, batch_size=1,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=1,
                                          shuffle=True, num_workers=2)

dataloaders={'train':trainloader, 'val':valloader, 'test':testloader}
dataset_sizes={'train':len(trainloader)*batch_size, 'val':len(valloader)*batch_size, 'test':len(testloader)*batch_size}


In [6]:
# inputs, labels = next(iter(testloader))
# inputs, labels = inputs.to(device), labels.to(device)
# output = model_ft(inputs)
# print("output=", output)
# output = F.softmax(output, dim=1)
# prediction_score, pred_label_idx = torch.topk(output, 1)
# pred_label_idx.squeeze_()
# print("class_names=",class_names)
# predicted_label = class_names[(pred_label_idx.item())]
# print("predicted_label=",predicted_label)
# print('Predicted:', predicted_label, '(', prediction_score .squeeze().item(), ')')

In [7]:
from captum.attr import IntegratedGradients
from captum.attr import NoiseTunnel
from matplotlib.colors import LinearSegmentedColormap
from captum.attr import visualization as viz
from PIL import Image

integrated_gradients = IntegratedGradients(model_ft)


In [8]:
def save_image_interpretation(model,dataset):
    was_training = model.training
    nb_image = 0
    fig = plt.figure()
    if dataset=='val':
        dataloader = valloader
    if dataset=='test':
        dataloader = testloader
    if dataset=='train':
        dataloader = trainloader
           
    default_cmap = LinearSegmentedColormap.from_list('custom blue', 
                                                 [(0, '#ffffff'),
                                                  (0.25, '#000000'),
                                                  (1, '#000000')], N=256)
    IMAGE_DIR = Path("/home/jfalck/PycharmProjects/drawinganalyses/drawinganalyses/image/Captum/" + dataset)
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloader):
            nb_image+=1
            inputs = inputs.to(device)
            labels = labels.to(device)
            print(labels)
            output = model(inputs)
            output = F.softmax(output, dim=1)
            prediction_score, pred_label_idx = torch.topk(output, 1)
            print(prediction_score, pred_label_idx)
            pred_label_idx.squeeze_()
            print("Images numéros:", nb_image, "sur ",len(dataloader))
            noise_tunnel = NoiseTunnel(integrated_gradients)
            attributions_ig_nt = noise_tunnel.attribute(inputs,
                                            nt_type='smoothgrad_sq', target=pred_label_idx.item())
            transposed_attr_ig_nt = np.transpose(attributions_ig_nt.squeeze().cpu().detach().numpy(), (1,2,0))
            transposed_image = np.transpose(inputs.squeeze().cpu().detach().numpy(), (1,2,0))
            fig, axis = viz.visualize_image_attr_multiple(transposed_attr_ig_nt,
                                      transposed_image,
                                      ["original_image", "heat_map"],
                                      ["all", "positive"],
                                      cmap=default_cmap,
                                      show_colorbar=True)
            fig.savefig(os.path.join(IMAGE_DIR,str(nb_image)+'.png'))
            
        model.train(mode=was_training)

In [9]:
save_image_interpretation(model_ft,dataset='val')

/home/jfalck/PycharmProjects/drawinganalyses/venv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/home/jfalck/PycharmProjects/drawinganalyses/venv/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc

tensor([0])
tensor([[0.5352]]) tensor([[0]])
Images numéros: 1 sur  40


KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>